# Ejercicio Sencillo Agent2Agent con Batman + RAG

## Objetivo
Implementar un flujo **agent2agent** minimalista y util en AI Engineering aplicado:
- **Agent 1 (Detective Retriever)**: recupera evidencia desde la base vectorial.
- **Agent 2 (Profesor Synthesizer)**: construye la respuesta final sustentada en contexto.

Si la respuesta no esta suficientemente fundamentada, el orquestador solicita un segundo ciclo de retrieval.

## Diagrama del ejercicio

```mermaid
flowchart TD
  A["Pregunta del usuario"] --> B["Detective Retriever Agent"]
  B --> C["Contexto recuperado"]
  C --> D["Profesor Synthesizer Agent"]
  D --> E["Evaluacion de grounding"]
  E -->|"bajo"| B
  E -->|"suficiente"| F["Respuesta final"]
```

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
import sys
import time
import pandas as pd

ROOT = Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from scripts.common import generate_answer
from scripts.evaluation import groundedness_score
from scripts.vector_store_lab import build_index_from_json

OUTPUTS_DIR = ROOT / 'outputs'
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)
DATA_PATH = ROOT / 'data' / 'batman_comics.json'

In [ ]:
db, chunks, index_stats, chunk_stats = build_index_from_json(
    json_path=DATA_PATH,
    persist_dir=OUTPUTS_DIR / 'chroma_agent2agent_batman',
    collection_name='agent2agent_batman',
    chunk_size=800,
    chunk_overlap=120,
    embedding_model='text-embedding-3-small',
)

print('Index stats:', index_stats)
print('Chunk stats:', chunk_stats)

In [ ]:
@dataclass
class DetectiveRetrieverAgent:
    vector_db: object
    embedding_model: str = 'text-embedding-3-small'

    def retrieve(self, query: str, k: int = 5) -> tuple[list[dict], str]:
        docs, provider = self.vector_db.query(
            query_text=query,
            n_results=k,
            embedding_model=self.embedding_model,
        )
        return docs, provider


@dataclass
class ProfesorSynthesizerAgent:
    model: str = 'gpt-5-mini'

    def answer(self, question: str, docs: list[dict], mode: str = 'normal') -> tuple[str, str]:
        contexts = [str(doc.get('text', '')) for doc in docs]
        if mode == 'estricto':
            system_prompt = (
                'Modo estricto de verificabilidad: responde solo con evidencia del contexto, '
                'no inventes datos, y cita [D#].'
            )
        else:
            system_prompt = (
                'Eres profesor de AI Engineering aplicado. Explica con claridad y rigor tecnico, '
                'limitandote al contexto recuperado y citando [D#].'
            )

        return generate_answer(
            query=question,
            contexts=contexts,
            model=self.model,
            system_prompt=system_prompt,
        )


class Agent2AgentOrchestrator:
    def __init__(self, retriever: DetectiveRetrieverAgent, synthesizer: ProfesorSynthesizerAgent) -> None:
        self.retriever = retriever
        self.synthesizer = synthesizer

    def run(self, question: str, k: int = 5, grounding_threshold: float = 0.18) -> dict:
        t0 = time.perf_counter()
        dialogue = []

        docs, retrieval_provider = self.retriever.retrieve(question, k=k)
        dialogue.append({
            'agent': 'DetectiveRetrieverAgent',
            'message': f'Recuperados {len(docs)} documentos con provider={retrieval_provider}',
        })

        answer, llm_provider = self.synthesizer.answer(question=question, docs=docs, mode='normal')
        score = groundedness_score(answer=answer, contexts=[doc.get('text', '') for doc in docs])
        dialogue.append({
            'agent': 'ProfesorSynthesizerAgent',
            'message': f'Borrador generado con provider={llm_provider} y groundedness={score}',
        })

        if score < grounding_threshold:
            refined_query = question + ' Enfocate en evidencia canonica concreta y eventos verificables.'
            docs_refined, retrieval_provider_2 = self.retriever.retrieve(refined_query, k=k + 2)
            dialogue.append({
                'agent': 'DetectiveRetrieverAgent',
                'message': (
                    f'Retrieval de refuerzo: {len(docs_refined)} docs con provider={retrieval_provider_2}'
                ),
            })
            docs = docs_refined
            answer, llm_provider = self.synthesizer.answer(question=question, docs=docs, mode='estricto')
            score = groundedness_score(answer=answer, contexts=[doc.get('text', '') for doc in docs])
            dialogue.append({
                'agent': 'ProfesorSynthesizerAgent',
                'message': f'Respuesta final regenerada con groundedness={score}',
            })

        latency = round(time.perf_counter() - t0, 4)
        return {
            'question': question,
            'answer': answer,
            'groundedness': score,
            'retrieved_docs': len(docs),
            'latency_seconds': latency,
            'llm_provider': llm_provider,
            'retrieval_provider': retrieval_provider,
            'dialogue': dialogue,
        }

In [ ]:
retriever_agent = DetectiveRetrieverAgent(vector_db=db, embedding_model='text-embedding-3-small')
synthesizer_agent = ProfesorSynthesizerAgent(model='gpt-5-mini')
orchestrator = Agent2AgentOrchestrator(retriever=retriever_agent, synthesizer=synthesizer_agent)

print('Agent2Agent pipeline listo.')

In [ ]:
query = 'Explica como Batman combina investigacion y estrategia para enfrentar amenazas como Bane y el Joker.'
result = orchestrator.run(question=query, k=5, grounding_threshold=0.18)

pd.DataFrame(result['dialogue'])

In [ ]:
print('Pregunta:')
print(result['question'])
print('\nRespuesta:')
print(result['answer'])
print('\nMetricas:')
print({
    'groundedness': result['groundedness'],
    'retrieved_docs': result['retrieved_docs'],
    'latency_seconds': result['latency_seconds'],
    'llm_provider': result['llm_provider'],
    'retrieval_provider': result['retrieval_provider'],
})

## Mini-practica guiada

Ejecuta estas consultas y compara el comportamiento del dialogo entre agentes:
1. `Que leccion deja Knightfall sobre delegacion y resiliencia?`
2. `Como se relaciona la filosofia de no matar con decisiones tacticas en Gotham?`
3. `Por que Batman es efectivo en la Liga de la Justicia sin superpoderes?`

In [ ]:
exercise_queries = [
    'Que leccion deja Knightfall sobre delegacion y resiliencia?',
    'Como se relaciona la filosofia de no matar con decisiones tacticas en Gotham?',
    'Por que Batman es efectivo en la Liga de la Justicia sin superpoderes?',
]

rows = []
for q in exercise_queries:
    run = orchestrator.run(question=q, k=5, grounding_threshold=0.18)
    rows.append({
        'question': q,
        'groundedness': run['groundedness'],
        'retrieved_docs': run['retrieved_docs'],
        'latency_seconds': run['latency_seconds'],
        'llm_provider': run['llm_provider'],
        'retrieval_provider': run['retrieval_provider'],
    })

exercise_df = pd.DataFrame(rows)
exercise_df

In [ ]:
csv_path = OUTPUTS_DIR / 'agent2agent_exercise_results.csv'
exercise_df.to_csv(csv_path, index=False)
print(f'Saved: {csv_path}')

## Cierre

Este ejercicio ilustra una idea importante: en muchos casos reales, un pipeline **agent2agent simple y auditable** aporta mas valor que una arquitectura multi-agente compleja.